# 1. Imports

In [1]:
from src.configuration import Configuration
from src.fetch_data import DataFetcher
from aggregate_data import DataAggregator
import pandas as pd
import time

# 2. Variables

In [2]:
config = Configuration('src/config.cfg')
fetcher = DataFetcher(config)
DATAAGGREGATOR = DataAggregator(config, fetcher)

25/10/15 19:49:35 WARN Utils: Your hostname, MacBook-Pro-Grzegorz.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
25/10/15 19:49:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/15 19:49:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 3. Data

## 3.1 Driver related data

In [ ]:
driver_last_5_races_result = DATAAGGREGATOR.get_last_races_result(5,"Race", "position")
time.sleep(3)
driver_last_5_quali_result = DATAAGGREGATOR.get_last_races_result(5,"Qualifying", "position")
time.sleep(3)
driver_last_5_avg_race_position = DATAAGGREGATOR.get_last_races_result(5, "Race", "avg")
time.sleep(3)
driver_last_5_std_race_position = DATAAGGREGATOR.get_last_races_result(5, "Race", "std")
time.sleep(3)
driver_last_5_avg_quali_position = DATAAGGREGATOR.get_last_races_result(5, "Qualifying", "avg")
time.sleep(3)
driver_last_5_std_quali_position = DATAAGGREGATOR.get_last_races_result(5, "Qualifying", "std")
time.sleep(3)
driver_points_gathered = DATAAGGREGATOR.get_racer_team_points("driver")[[""]]
time.sleep(5)
driver_gap_to_teammate = DATAAGGREGATOR.calculate_gap_to_teammate()
time.sleep(5)
driver_gap_to_leader = DATAAGGREGATOR.calculate_gap_to_leader()

/Users/grzegorznaporowski/Desktop/Portfolio/F1/aggregate_data.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_filtered["points_diff"] = merged_filtered["points_gained"] - merged_filtered["points_gained_other"]


## 3.2 Team related data

In [4]:
team_points_gathered = DATAAGGREGATOR.get_racer_team_points("team")

# Joined Data

In [18]:
driver_last_5_races_result.merge(
    driver_last_5_quali_result,
    on = ["key","driver_number"],
    how = "inner",
    suffixes=["_race","_quali"]
).merge(
    driver_last_5_avg_race_position,
    on = ["driver_number", "key"]
).merge(
    driver_last_5_std_race_position,
    on = ["driver_number", "key"]
).merge(
    driver_last_5_avg_quali_position,
    on = ["driver_number", "key"]
).merge(
    driver_last_5_std_quali_position,
    on = ["driver_number", "key"]
).merge(
    driver_points_gathered[["driver_number","key","points_gained"]],
    on = ["driver_number","key"]
).merge(
    driver_gap_to_teammate,
    on = ["driver_number","key"]
).merge(
    driver_gap_to_leader,
    on = ["driver_number","key"]
).merge(
    team_points_gathered,
    on = ["driver_number","key"]
)

,driver_number,position_race,key,last_race_pos_1,last_race_pos_2,last_race_pos_3,last_race_pos_4,last_race_pos_5,position_quali,last_qualifying_pos_1,...,last_qualifying_pos_5,avg_last_5_race,std_last_5_race,avg_last_5_qualifying,std_last_5_qualifying,points_gained,points_diff,gap_to_leader,team_name,team_points_gained
0,1,1.0,Monaco2023,1.0,2.0,1.0,2.0,1.0,1.0,1.0,...,9.0,1.4,0.489898,5.6,5.571355,119.0,22.0,0.0,Red Bull Racing,216.0
1,1,1.0,Barcelona2023,2.0,1.0,2.0,1.0,1.0,1.0,15.0,...,1.0,1.4,0.489898,5.6,5.571355,145.0,36.0,0.0,Red Bull Racing,254.0
2,1,1.0,Montréal2023,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.2,0.400000,2.8,3.124100,170.0,52.0,0.0,Red Bull Racing,288.0
3,1,1.0,Spielberg2023,2.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.2,0.400000,2.8,3.124100,196.0,63.0,0.0,Red Bull Racing,329.0
4,1,1.0,Silverstone2023,1.0,1.0,1.0,1.0,1.0,1.0,9.0,...,1.0,1.0,0.000000,2.6,3.200000,222.0,81.0,0.0,Red Bull Racing,363.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,87,21.0,Budapest2025,17.0,11.0,11.0,11.0,11.0,11.0,15.0,...,12.0,12.2,2.400000,12.8,2.638181,6.0,-17.0,257.0,Haas F1 Team,29.0
1100,87,6.0,Zandvoort2025,11.0,11.0,11.0,11.0,21.0,19.0,14.0,...,11.0,13.0,4.000000,12.0,2.449490,14.0,-10.0,274.0,Haas F1 Team,38.0
1101,87,12.0,Monza2025,11.0,11.0,11.0,21.0,6.0,11.0,15.0,...,19.0,12.0,4.898979,13.0,3.741657,14.0,-10.0,289.0,Haas F1 Team,38.0
1102,87,12.0,Baku2025,11.0,11.0,21.0,6.0,12.0,15.0,8.0,...,11.0,12.2,4.874423,12.2,3.655133,14.0,-10.0,289.0,Haas F1 Team,38.0


In [17]:
team_points_gathered

,key,driver_number,team_name,team_points_gained
0,Sakhir2023,77,Alfa Romeo,4.0
1,Sakhir2023,24,Alfa Romeo,4.0
2,Jeddah2023,24,Alfa Romeo,4.0
3,Jeddah2023,77,Alfa Romeo,4.0
4,Melbourne2023,24,Alfa Romeo,6.0
...,...,...,...,...
1272,Monza2025,55,Williams,83.0
1273,Baku2025,55,Williams,98.0
1274,Baku2025,23,Williams,98.0
1275,Marina Bay2025,55,Williams,99.0
